### Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/2023_AN2DL/Homework

### Import libraries

In [ ]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [ ]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

In [ ]:
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler

### Load data

In [ ]:
dataset = np.load('training_data.npy')
# Convert into pandas dataframe
dataset = pd.DataFrame.from_dict(dataset)
print(dataset.shape)

In [ ]:
transposed_dataset = dataset.T
print(transposed_dataset.shape)

In [ ]:
selected_columns = transposed_dataset.iloc[:, :500]
print(selected_columns.shape)

# Manage time series

In [ ]:
window=200
stride=5
direct_telescope=18

In [ ]:
def build_sequences(df, window=200, stride=5, telescope=18):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    X_train = []
    y_train = []
    temp_df = np.copy(df)
    padding_check = len(df) % window
    series_count = temp_df.shape[1]

    if padding_check != 0:
        # Compute padding length
        padding_len = window - len(df) % window
        padding = np.zeros((padding_len, temp_df.shape[1]), dtype='float32')
        temp_df = np.concatenate((padding, df))
        assert len(temp_df) % window == 0

    # Loop through each series
    for series_idx in range(series_count):
        series_data = temp_df[:, series_idx]  # Extract one series at a time

        for idx in np.arange(0, len(series_data) - window - telescope, stride):
          window_data = series_data[idx:idx+window]
          X_train.append(window_data)

          target_values = series_data[idx + window: idx + window + telescope]
          y_train.append(target_values)

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_train = X_train.reshape(-1, window, 1)
    y_train = y_train.reshape(-1, telescope, 1)

    return X_train, y_train

In [ ]:
X_train, y_train = build_sequences(selected_columns, window, stride, direct_telescope)
X_train.shape, y_train.shape

# Model

In [ ]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 64
epochs = 200

In [ ]:
print(input_shape)
print(output_shape)

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Ensure the input time steps are at least as many as the output time steps
    assert input_shape[0] >= output_shape[0], "For this exercise we want input time steps to be >= of output time steps"

    # Define the input layer with the specified shape
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    # Add a Bidirectional LSTM layer with 64 units
    x = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True, name='lstm'), name='bidirectional_lstm')(input_layer)

    # Add a 1D Convolution layer with 128 filters and a kernel size of 3
    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv')(x)

    # Add a flatten layer
    x = tf.keras.layers.Flatten()(x)

    # Add a Dense layer to match the required output shape
    output_layer = tf.keras.layers.Dense(output_shape[0] * output_shape[1])(x)

    # Reshape to match the desired output shape
    x = tf.keras.layers.Reshape(output_shape, name='reshape')(x)

    # Remove the last dimension
    output_layer = tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=-1))(x)

    # Construct the model by connecting input and output layers
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')

    # Compile the model with Mean Squared Error loss and Adam optimizer
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())

    return model

In [ ]:
model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True, show_shapes=True)

In [ ]:
# Train the model
history = model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.2,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=12, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=10, factor=0.1, min_lr=1e-5)
    ]
).history

In [ ]:
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [ ]:
model.save('18_telescope_lastmodel')
model = tfk.models.load_model('18_telescope_lastmodel')